In [0]:
from pyspark.sql.functions import current_timestamp, col, flatten, array, expr, lit
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType
from datetime import date
import json
from time import time

In [0]:
# https://learn.microsoft.com/en-us/azure/databricks/jobs/file-arrival-triggers
# Configuration
# TODO need to get the most recent file names?
file_location = "/Volumes/runescape/00_landing/data_sources/latest_prices_transformed/" # The same URL configured for the file arrival trigger.
checkpoint_location = "/Volumes/runescape/00_landing/data_sources/latest_prices_transformed_checkpoint/" # a separate URL (outside `file_location`) used to store the Auto Loader checkpoint, which enables exactly-once processing.

#Schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("price", IntegerType(), True),
    StructField("time", LongType(), True),
    StructField("highorlow", StringType(), True)
])

# Read all the csv files written atomically in a directory
df = spark \
    .readStream \
    .schema(schema) \
    .format("parquet") \
    .load(file_location)

#df.write.mode("append").saveAsTable("runescape.01_bronze.latest_prices_raw")

df.writeStream \
    .format("console") \
    .trigger(availableNow=True) \
    .start()

In [0]:
%skip
# https://learn.microsoft.com/en-us/azure/databricks/jobs/file-arrival-triggers
# Configuration
# TODO need to get the most recent file names?
file_location = "/Volumes/runescape/00_landing/data_sources/latest_prices_transformed/" # The same URL configured for the file arrival trigger.
checkpoint_location = "/Volumes/runescape/00_landing/data_sources/latest_prices_transformed_checkpoint/" # a separate URL (outside `file_location`) used to store the Auto Loader checkpoint, which enables exactly-once processing.




#Schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("price", IntegerType(), True),
    StructField("time", LongType(), True),
    StructField("highorlow", StringType(), True)
])

# Use Auto Loader to discover new files.
streamingQuery = (spark \
    .readStream \
    .schema(schema) \
    .format("parquet") \
    .load(file_location) \
    .writeStream \
    .outputMode("append") \
    .option("checkpointLocation", checkpoint_location) \
    .trigger(availableNow=True) \
    .toTable("runescape.01_bronze.latest_prices_raw") # TODO the data does not being written to the table
)

In [0]:
%skip
# https://learn.microsoft.com/en-us/azure/databricks/jobs/file-arrival-triggers
# Configuration
file_location = "/Volumes/runescape/00_landing/data_sources/latest_prices_transformed/" # The same URL configured for the file arrival trigger.
checkpoint_location = "/Volumes/runescape/00_landing/data_sources/latest_prices_transformed_checkpoint/" # a separate URL (outside `file_location`) used to store the Auto Loader checkpoint, which enables exactly-once processing.
sink_table = "runescape.01_bronze.latest_prices_raw" # Delta table to write to

#Schema
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("price", IntegerType(), True),
    StructField("time", LongType(), True),
    StructField("highorlow", StringType(), True),
    StructField("scanTime", LongType(), True)
])

# Use Auto Loader to discover new files.
streamingQuery = (spark \
    .readStream
    .schema(schema)
    .format("console")
    .load(file_location)
    .writeStream
    .option("checkpointLocation", checkpoint_location) \
    
    .load()
)

In [0]:
%skip
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

In [0]:
%skip
df_itemprice.display()

Should the data only write new values to the table below?